# Recurrent Neural Network

This notebook will read in the merged dataset of session behavior and track IDs, preprocess the data and perform an RNN model using an LSTM layer. 

The RNN model was chosen here because the sequence of the rows is important. The dataset is composed of several unique session IDs, each session consists of 20 rows. The data is reshaped during preprocessing to group the sessions. The LSTM layer is capable of learning long term dependencies, but predicting the following sequence based on the recent instances. 

### Import libraries and read in the dataset

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import models
from keras import layers
from keras import optimizers

#from src folder
from preprocessing import *

In [7]:
df = pd.read_csv('tracks_session_clean.csv', index_col=0)

##There is a variety of session lengths. The sessions with 20 instances will be used.

df = df.loc[df['session_length'] == 20]

### Drop columns

To reduce making assumptions about the metadata, the acoustic vector columns were dropped. There are still plenty of music features available in the dataset that are more interpretable. Other dropped columns contained multicolinearity from the data exploration.

In [8]:
drop_col = ['session_length', 'date', 'track_id', 'hist_user_behavior_reason_start', 'beat_strength', 'dyn_range_mean',
'acoustic_vector_0', 'acoustic_vector_1', 'acoustic_vector_2', 'acoustic_vector_3', 'acoustic_vector_4', 'acoustic_vector_5', 'acoustic_vector_6', 'acoustic_vector_7']

df = df.drop(labels=drop_col, axis=1).reset_index(drop=True)

### Preprocessing

Takes in a dataframe. Returns `train_features`, `test_features`, `train_labels`, and `test_labels`. It also scales the features, then reshapes them to fit the 3D LSTM layer.

In [9]:
train_features, test_features, train_labels, test_labels = preprocess(df)

In [ ]:
print(train_features.shape)
print(train_labels.shape)
print(test_features.shape)
print(test_labels.shape)

### Build & train model

In [6]:
model = keras.Sequential()
model.add(layers.LSTM(16, activation='relu', return_sequences=True))
model.add(layers.Dropout(.2))
model.add(layers.Dense(16, activation='tanh'))
model.add(layers.Dense(8, activation='relu'))

#Binary outcome
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', metrics='accuracy', optimizer='Adam')

In [7]:
model.fit(train_features, train_labels, epochs=100, verbose=1, shuffle=False)

Epoch 1/100
110/110 [==============================] - 6s 11ms/step - loss: 0.6887 - accuracy: 0.5491
Epoch 2/100
110/110 [==============================] - 2s 16ms/step - loss: 0.6800 - accuracy: 0.5636
Epoch 3/100
110/110 [==============================] - 2s 23ms/step - loss: 0.6758 - accuracy: 0.5746
Epoch 4/100
110/110 [==============================] - 3s 24ms/step - loss: 0.6720 - accuracy: 0.5825
Epoch 5/100
110/110 [==============================] - 2s 19ms/step - loss: 0.6692 - accuracy: 0.5897
Epoch 6/100
110/110 [==============================] - 2s 20ms/step - loss: 0.6667 - accuracy: 0.5915
Epoch 7/100
110/110 [==============================] - 2s 22ms/step - loss: 0.6651 - accuracy: 0.5948
Epoch 8/100
110/110 [==============================] - 2s 18ms/step - loss: 0.6630 - accuracy: 0.5969
Epoch 9/100
110/110 [==============================] - 2s 14ms/step - loss: 0.6619 - accuracy: 0.6014
Epoch 10/100
110/110 [==============================] - 2s 17ms/step - loss: 0.660

In [8]:
model.evaluate(test_features, test_labels)

49/49 [==============================] - 1s 4ms/step - loss: 0.7262 - accuracy: 0.5708


[0.7261636853218079, 0.5707960724830627]

### Interpretation